In [ ]:
import pathlib
import sys

In [ ]:
import joblib
import kaggle
import pandas as pd

In [ ]:
src_dir = "../../src"
description = ""

In [ ]:
sys.path.append(src_dir)

In [ ]:
from package.constants import *

In [ ]:
sample_submission = pd.read_csv(sample_submission_path)

In [ ]:
prediction = pd.read_parquet(prediction_path)

In [ ]:
d = prediction["d"].str[2:]
d = d.astype("int")
d -= train_days + 1

is_evaluation = d >= test_days
d %= test_days

prediction["d"] = d
prediction.loc[is_evaluation, "id"] = prediction.loc[is_evaluation, "id"].str.replace(
    "_validation", "_evaluation"
)

In [ ]:
submission = pd.pivot(prediction, index="id", columns="d", values=target)

In [ ]:
submission.reset_index(inplace=True)

In [ ]:
submission.columns = ["id"] + [f"F{i + 1}" for i in range(test_days)]

In [ ]:
submission = pd.merge(sample_submission["id"], submission, how="left")

In [ ]:
submission.to_csv(submission_accuracy_path, index=False)

In [ ]:
kaggle.api.competition_submit(
    submission_accuracy_path,
    message=description,
    competition="m5-forecasting-accuracy",
)